In [ ]:
import pandas as pd
import numpy as np
import sidetable
import pymongo

import folium
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append("../")
import src.creacion_columnas as cc

### Manipulación de columnas

In [ ]:
df3 = pd.read_csv ("Data/03. Datos_mergeados_.csv")
df3.head(2)

Vamos a eliminar la columna "Unnamed: 0", que luego nos generará problemas con la identificación de duplicados

In [ ]:
"""df3.pop('Unnamed: 0')"""

Vamos establecer únicamente dos opciones para la columna AD_Situation: New, old. 

Vamos a quedarnos únicamente con WA, OR o CA en la columna "State".

Para ello, hemos creado una función donde haremos un split por espacio y nos quedaremos con la primera parte del string

In [ ]:
"""def quitar_horas(x): 
    try:
        return x.split()[0]
    except: 
        return x"""

In [ ]:
df3["AD_Situation"] = df3["AD_Situation"].apply(cc.quitar_horas)
df3["State"] = df3["State"].apply(cc.quitar_horas)
df3.head(2)

Vamos a utilizar una función que hemos creado para cambiar de sqft a m2, y lo vamos a aplicar sobre la columna "Building space". El objetivo es dividir la columna "Building space Measruing" por 10.764:

In [ ]:
"""def calcular_m2 (x):
    div = x/10.764
    return round(div, 0)"""

In [ ]:
"""df3["Building space"] = df3.apply(lambda datos: cc.calcular_m2(datos["Building space"]), axis = 1)
df3.head(2)"""

Vamos a cambiar la unidad de medida de "sqft" a "m2" sobre la columna "Measruing unit".

Vamos a cambiar la columna "Bulding space" para que esté en m2 en vez de sqft. Para ello lo vamos a dividir por 10.764.

Vamos a crear una columna nueva: "Price/m2", donde vamos dividir la columna "Price" por la columna "Building space". 

In [ ]:
"""def calcular_precio_por_m2 (x, y):
    div = x/y
    return round(div)"""

In [ ]:
"""df3["Price/m2"] = df3.apply(lambda df3: cc.calcular_precio_por_m2(df3["Price"], df3["Building space"]), axis = 1)
df3['Measuringg unit']= df3['Measruing unit'].replace("sqft","m2")
df3.head(2)"""

In [ ]:
df3['Measruing unit']= df3['Measruing unit'].replace("sqft","m2")
df3['Building_space']= round(df3['Building space']/10.764, 0)
df3['Price/m2']= round((df3['Price'])/(df3['Building space']), 0)
df3.head(2)

Vamos a reescriir la columna "State", para que muestre únicamente los 3 Estado: California, Oregón, Washington:

In [ ]:
"""df3["State"] = df3.State.str.extract("(\w+)", expand=True)
df3.head(2)"""

In [ ]:
""""def cambiar(x):
    if x == "WA":
        return x.replace("WA", "Washington")

    elif  x == "OR":
        return x.replace("OR", "Oregón")

    else:
        return x.replace("CA", "California")"""

In [ ]:
df3["State"] = df3.apply(lambda datos: cc.cambiar(datos["State"]),axis = 1)

Vamos a crear cuatro columnas nuevas, "dirección", "ciudad1", "estado1, "None" a partir de la columna "Address". En las nuevas columnas vamos a mostrar la información de "Address" spliteada por ",":

In [ ]:
df3[["direccion1", "ciudad1", "código_postal1", "None1"]] = df3["Address"].str.split(",",  expand=True)
df3.head(2)

In [ ]:
df3["Direction"] = df3["direccion1"] + "," + df3["ciudad1"]
df3.head(2)

In [ ]:
df3["Postal_code"] = df3.código_postal1.str.extract("(\d+)", expand=True)
df3.head(2)

Vamos a eliminar las columna que no nos interesan por estar la información "duplicada"

In [ ]:
eliminar_columns = ['Address', 'Unnamed: 0','direccion1', 'Measruing unit', 'ciudad1', 'código_postal1', 'None1', 'Building space', 'Measruing unit']

In [ ]:
df3_limpio = df3.drop(eliminar_columns, axis = 1)
df3_limpio.head(2)

Problema: Estandarización del tipado de las columnas.
Vamos a ordenar las columnas de nuestro DataFrame para que tenga algo más de sentido:

In [ ]:
df3_limpio = df3_limpio.reindex(columns=['Price', 'Currency', 'AD_Situation', 'Rooms', 'Bath', 'Building_space', 'Measuring unit', 'Price/m2', 'Brokered_by', 'Direction', 'City', 'State', 'Postal_code',
       'Índice_poder_adqui', 'Índice_seguridad', 'Índice_sanidad',
       'Índice_clima', 'Índice_costo_vida',
       'Relación_precio_ingresos_propiedades', 'Índice_costo_desplazamiento',
       'Índice_contaminación', 'Índice_calidad_vida'])
df3_limpio.head(2)

Problema: Estandarización del tipado de las columnas.
A continuación, vamos a homogeneizar las columnas: pasar los nombres de las columnas a minusculas y reemplazar los espacios por "_":

In [ ]:
df3_limpio.columns

In [ ]:
diccionario = {}
for col in df3_limpio.columns:
    diccionario[col] = col.lower().strip().replace(" ", "_")

In [ ]:
nuevas_columnas = {col : col.lower().strip().replace(" ", "_") for col in df3_limpio.columns}
nuevas_columnas

In [ ]:
df3_limpio.rename(columns = nuevas_columnas, inplace =True)
df3_limpio.head(2)

### Tipo de Datos

Problema: La columna "baños" está en formato "float". Debemos convertirlas a integer. Las columans "índice_poder_adqui', 'índice_seguridad', 'índice_sanidad', 'índice_clima', 'índice_costo_vida', 'relación_precio_ingresos_propiedades', 'índice_costo_desplazamiento', 'índice_contaminación', 'índice_calidad_vida' están en formato "string"... Tendremos que hacer algo con ellas más adelante. Debemos convertirlas a integer:

In [ ]:
df3_limpio.dtypes

In [ ]:
df3_limpio["bath"] = df3_limpio["bath"].round(1).astype(int)
df3_limpio.head(2)

In [ ]:
x = ['índice_poder_adqui', 'índice_seguridad',
       'índice_sanidad', 'índice_clima', 'índice_costo_vida',
       'relación_precio_ingresos_propiedades', 'índice_costo_desplazamiento',
       'índice_contaminación', 'índice_calidad_vida']

def puntos(df,x):
    for i in x:
        
        df3_limpio[i] = df3_limpio [i].str.replace(",","")
        df3_limpio[i] = pd.to_numeric(df[i], errors = "coerce").astype("Int64")


In [ ]:
puntos(df3_limpio, x)

In [ ]:
df3_limpio.dtypes

### Duplicados

Problema: Duplicado. En el análisis exploratorio identificamos 2 duplicados:

In [ ]:
df3_limpio.duplicated().sum()

In [ ]:
df3_limpio[df3_limpio.duplicated() == True]

Una vez identificamos las filas duplicadas, procedemos a su eliminación:

In [ ]:
df3_limpio.drop_duplicates(inplace = True)

In [ ]:
df3_limpio.duplicated().sum()

### Principales estadísticos

In [ ]:
df3_limpio.describe().T

In [ ]:
df3_limpio.describe(include = "object").T

In [ ]:
df_line = pd.DataFrame(df3_limpio.groupby([df3_limpio['city']])["price"].mean())

In [ ]:
df_line.head(3)

In [ ]:
ax = plt.axes() # para poder cambiar el estilo de los ejes

ax.set_facecolor("white") # poner el fondo en blanco

ax.spines['bottom'].set_color('black') # color del eje x
ax.spines['top'].set_color('white') # color del eje de arriba
ax.spines['right'].set_color('white') # color del eje de la derecha
ax.spines['left'].set_color('black') # color del eje y

plt.plot(df_line['city'], # eje x
         df_line["price"], # eje y
         color = "black", # para cambiar el color de la línea
         linewidth = 2, # para cambiar el grosor de la línea
         marker = "o") # para poner los marcadores en nuestra línea

plt.xlabel("ciudad") # para poner etiqueta en el eje x
plt.ylabel("precio") # para poner etiqueta en el eje y


plt.show();

In [ ]:
sns.scatterplot(data=df3_limpio, x='índice_poder_adqui', y='price/m2',
                s = 100); # tamaño de los marcadores

### Relación Espacio de vivienda y precio

In [ ]:
sns.scatterplot(data=df3_limpio, x='building_space', y='price',
                  hue = "city",
                s = 100); # tamaño de los marcadores

In [ ]:
df_pie = pd.DataFrame(df3_limpio.groupby(['city'])
                      ["price/m2"].mean()).reset_index()

df_pie.head()

In [ ]:
df_pie_1 = pd.DataFrame(df3_limpio.groupby(['city'])
                      ["índice_poder_adqui"].mean()).reset_index()

df_pie_1.head()

In [ ]:
df_pie_1 = pd.DataFrame(df3_limpio.groupby(['city'])
                      ["índice_seguridad"].mean()).reset_index()

df_pie_1.head()

In [ ]:
sns.scatterplot(data=df3_limpio, x='state', y='índice_contaminación',
                s = 100); # tamaño de los marcadores

In [ ]:
sns.scatterplot(data=df3_limpio, x='índice_contaminación', y='índice_calidad_vida',
                s = 100); # tamaño de los marcadores

In [ ]:
sns.scatterplot(data=df3_limpio, x='relación_precio_ingresos_propiedades', y='price',
                s = 100); # tamaño de los marcadores

In [ ]:
sns.scatterplot(data=df3_limpio, x='state', y='índice_calidad_vida',
                s = 100); # tamaño de los marcadores

In [ ]:
sns.scatterplot(data=df3_limpio, x='sanidad', y='índice_contaminación',
                s = 100); # tamaño de los marcadores

In [ ]:
sns.scatterplot(data=df3_limpio, x='building_space', y='price',
                s = 100); # tamaño de los marcadores

In [ ]:
def explorar(df, ciudad):
    city = df[df["city"] == f"{ciudad}"]
    print("los principales estadisticos son: ")
    print(city['price/m2'].max())
    print("-----------")
    

In [ ]:
def explorar_min(df, ciudad):
    city = df[df["city"] == f"{ciudad}"]
    print("los principales estadisticos son: ")
    print(city['price/m2'].min())
    print("-----------")

In [ ]:
def explorar_v2(df, ciudad):
    city = df[df["city"] == f"{ciudad}"]
    print("los principales estadisticos son: ")
    print df3_limipio().T

In [ ]:
explorar(df3_limpio, "Irvine")

In [ ]:
explorar_min(df3_limpio, "Irvine")

In [ ]:
df3_limpio.groupby(["state", 'city'])["price/m2"].mean()

In [ ]:
df3_limpio.groupby(["state", 'city'])["price/m2"].max()

In [ ]:
df3_limpio.groupby(["state", 'city'])["price/m2"].min()

In [ ]:
df3_limpio.groupby(["state", 'city'])["índice_poder_adqui"].mean().max()

In [ ]:
df3_limpio.groupby(["state", 'city'])["price/m2"].mean()

In [ ]:
df3_limpio.groupby(['city'])["price/m2"].max()

In [ ]:
df3_limpio.groupby(['city'])["price/m2"].describe()

In [ ]:
df3_limpio.groupby(["state", 'city'])["price/m2"].min()

In [ ]:
df3_limpio.groupby(["state", 'city'])["price"].min()

In [ ]:
df3_limpio.groupby(["state", 'city'])["price"].mean().round()

In [ ]:
df3_limpio.groupby(["state"])["price"].min()

In [ ]:
df3_limpio.groupby(["state"])["price"].max().round(0)

In [ ]:
df3_limpio.groupby(["state"])["price"].mean().round(0)

In [ ]:
df5 = pd.DataFrame(df3_limpio.groupby(["state", 'city','postal_code'])["price/m2"].mean())
df5

In [ ]:
df4 = pd.DataFrame(df3_limpio.groupby(["state", 'city','postal_code'])["price/m2"].max())
df4

In [ ]:
df4.shape

In [ ]:
df3_limpio.groupby(["state", 'city'])["price/m2"].min()

### Valores Nulos

In [ ]:
df3_limpio.isnull().sum()

In [ ]:
df3_limpio["postal_code"] = df3_limpio["postal_code"].fillna(0)

In [ ]:
df3_limpio.isnull().sum()

In [ ]:
df3_limpio["postal_code"].dtypes

In [ ]:
df3_limpio["postal_code"] = df3_limpio["postal_code"].astype(int) 

In [ ]:
df3_limpio["postal_code"].dtypes

### Geopy

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
locator = Nominatim(user_agent= "myGeocoder")

In [ ]:
location = locator.geocode ("98516")
print (location)

In [ ]:
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

In [ ]:
location.latitude

In [ ]:
"""lista_codigos_postales=list(df3_limpio["postal_code"])"""

In [ ]:
"""for i in lista_codigos_postales:
    print(latitud_v2(i))"""

In [ ]:
def latitud_v2 (x):
    try:
        
        location = locator.geocode (x)
        return location.latitude
        
    except:
        return None


In [ ]:
def longitud_v2 (x):
    try:
        
        location = locator.geocode (x)
        return location.longitude
        
    except:
        return None


In [ ]:
df3_limpio["latidud_"] = df3_limpio.apply(lambda datos: latitud_v2(
                                    datos["postal_code"]), axis = 1)


In [ ]:
df3_limpio["longitud_"] = df3_limpio.apply(lambda datos: longitud_v2(
                                    datos["postal_code"]), axis = 1)


In [ ]:
mapa=folium.Map(location=[47.134266, -122.734934],  # la posicion
                tiles='openstreetmap',            # tipo de mapa, theme
                zoom_start=10)                    # zoom inicial

mapa

In [ ]:
for index, row in df.iterrows(): # método para iterar por los dataframes

cities = ['Olympia', 'Seattle', 'Des Moines', 'Bellevue', 'Redmond', 'Tacoma', 'Vancouver', 'Long Beach', 'Pasadena', 'Anaheim', 'Irvine', 'Portland']
    for i in cities:
        try:
                row['city'] == (i) :

        return folium.Marker([row["latitud_"], row['longitud_']],
                              radius=10, 
                              tooltip = row[['is_happy_customer', 'city']],
                              icon=folium.Icon(icon = "cloud", color = "red")).add_to(mapa)
        except:
            None

In [ ]:
for index, row in df.iterrows(): # método para iterar por los dataframes
    if row['city'] == "Olympia":
    
        folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10, 
                    tooltip = row[['city']],
                    icon=folium.Icon(icon = "cloud", color = "red")).add_to(mapa)
    elif row['city']  == "Seattle":
        folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10, 
                    tooltip = row['city'],
                    icon=folium.Icon(icon = "cloud", color = "orange")).add_to(mapa)
        
    elif row['city']  == "Des Moines":
        folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10, 
                    tooltip = row['city'],
                    icon=folium.Icon(icon = "cloud", color = "black")).add_to(mapa)   
    elif row['city']  == "Bellevue":
        folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10, 
                    tooltip = row['city'],
                    icon=folium.Icon(icon = "cloud", color = "yellow")).add_to(mapa)
        
    elif row['city']  == "Redmond":
            folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10, 
                    tooltip = row['city'],
                    icon=folium.Icon(icon = "cloud", color = "green")).add_to(mapa)  
    elif row['city']  == "Tacoma":
        folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10, 
                    tooltip = row['city'],
                    icon=folium.Icon(icon = "cloud", color = "purple")).add_to(mapa)
        
    elif row['city']  == "Vancouver":
        folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10, 
                    tooltip = row['city'],
                    icon=folium.Icon(icon = "cloud", color = "blue")).add_to(mapa) 
    
    elif row['city']  == "Long Beach":
        folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10, 
                    tooltip = row['city'],
                    icon=folium.Icon(icon = "cloud", color = "grey")).add_to(mapa)
        
    elif row['city']  == "Pasadena":
         folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10, 
                    tooltip = row['city'],
                    icon=folium.Icon(icon = "cloud", color = "pink")).add_to(mapa)  
    elif row['city']  == "Anaheim":
        folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10, 
                    tooltip = row['city'],
                    icon=folium.Icon(icon = "cloud", color = "violet")).add_to(mapa)
        
    elif row['city']  == "Irvine":
            folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10, 
                    tooltip = row['city'],
                    icon=folium.Icon(icon = "cloud", color = "plum")).add_to(mapa
    else:
        folium.Marker([row["latitud_"], row['longitud_']],
                    radius=10, 
                    tooltip = row['city'],
                    icon=folium.Icon(icon = "cloud", color = "brown")).add_to(mapa)

In [ ]:
df3_limpio["city"].unique()